In [1]:
import numpy as np
import mesh_to_sdf
import trimesh
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import tensorflow as tf
import mcubes
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
mesh = o3d.io.read_triangle_mesh("monkj.obj")
bounds = mesh.get_axis_aligned_bounding_box()
extent = np.max(bounds.get_extent(), axis=-1)
mesh.scale(1. / extent, center=bounds.get_center())

TriangleMesh with 3229 points and 1149 triangles.

In [3]:
verts = np.array(mesh.vertices)
print(verts.min(), verts.max())

-0.49999999999999994 0.49999999999999994


In [4]:
N = 250000
pcd = mesh.sample_points_uniformly(N)
pcd.colors = o3d.utility.Vector3dVector(np.random.uniform(0, 1, size=(N, 3)))
o3d.visualization.draw_geometries([pcd])

In [15]:
octree = o3d.geometry.Octree(max_depth=3)
octree.convert_from_point_cloud(pcd, size_expand=0.01)
o3d.visualization.draw_geometries([octree])

In [21]:
bounds = []
def f(n, ni, b=bounds):
    if ni.depth == 3:
        b.append(ni.origin)
octree.traverse(f)

In [22]:
bounds = np.array(bounds)
bounds.shape
bounds_pcd = o3d.geometry.PointCloud(o3d.utility.Vector3dVector(bounds))

0.125